In [ ]:
# Import libraries

In [ ]:
from ipywidgets import Layout, Button, VBox, Label, HBox
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from ipywidgets import Button, HBox, VBox, Layout , FileUpload
import numpy as np
import xml.etree.ElementTree as ET
from skimage import io, feature, morphology
from skimage import measure
from  scipy import ndimage as nd
from skimage.color import label2rgb
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.measure import label, regionprops, points_in_poly,  subdivide_polygon, approximate_polygon, find_contours, regionprops_table
from skimage import data, filters, measure, morphology
from skimage.morphology import skeletonize
from PIL import Image
import base64
from io import BytesIO
import random
import string
import datetime
from scipy import interpolate


In [ ]:
# Interaction callbacks

In [ ]:
# Callback fonction that upload and save images to png and return completion into htlm text          
def response(change):
    
    global thumbnail 
    global listofimage
    
    #Upload content, update the list of image and save to myview1  
    
    uploaded_filename = next(iter(Uploader.value))
    listofimage = listofimage + [uploaded_filename]
    
    content = Uploader.value[uploaded_filename]['content']
    with open('myview1.png', 'wb') as f: f.write(content)
    text = Image.open('myview1.png')
 
    #operation on images
    if Resize.value == True: # set the base width of the result
        basewidth = 300
        wpercent = (basewidth/float(text.size[0]))
        hsize = int((float(text.size[1])*float(wpercent)))
        # resize image
        text = text.resize((basewidth,hsize), Image.ANTIALIAS)
    else:
        pass
    
    if BlackandWhite.value == True: # set the threshold 
        thresh = 254
        fn = lambda x : 255 if x > thresh else 0
        text = text.convert('L').point(fn, mode='1')
    else:
        pass
    
    #full screen image
    pil_img = Image.fromarray(np.flipud(text)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
    
    imagetodisplay = go.Image(source = base64_string)
    #skeleton = skeletonize(base64_string)
    #skeletontodiplay = go.Image(source = skeleton)
    
    with lol.batch_update():
        lol.data = []
        #details1.value = uploaded_filename+" has been uploaded."
        lol.add_trace(imagetodisplay)
        #lol.add_trace(skeletontodiplay)
        details1.value = uploaded_filename+" has been uploaded"
        if Resize.value == True:
            details1.value = details1.value + ' and resized'
        details1.value = details1.value + '.'
        
    
    # thumbnail display
    
    basewidth = 300
    wpercent = (basewidth/float(text.size[0]))
    hsize = int((float(text.size[1])*float(wpercent)))
    # resize image
    text = text.resize((basewidth,hsize), Image.ANTIALIAS)
    
    #thresh = 254
    #fn = lambda x : 255 if x > thresh else 0
    # convert to b&w
    #text = text.convert('L').point(fn, mode='1')
    
    pil_img = Image.fromarray(np.flipud(text)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
    
    imagetodisplay = go.Image(source = base64_string)
    localbutton = []
    localbutton = widgets.ToggleButton(description = uploaded_filename)
    localbutton.observe(changeimage)
    
    layout = go.Layout(title='',
                    #width=480,
                    margin=dict(
                                l=1,
                                r=1,
                                b=1,
                                t=8,
                                pad=4
                                ),
                    font=dict(
                                family="Courier New, monospace",
                                size=8,
                                #color="RebeccaPurple"
                    )
                   )
    
    #config = {'displayModeBar': False}
    
    thumbnail = go.FigureWidget(layout=layout)
    
    #thumbnail.update_layout()

    # define root figure layout
    thumbnail.update_xaxes(autorange=True)
    thumbnail.update_yaxes(autorange=True)
    thumbnail.update_xaxes(visible=False)  
    thumbnail.update_yaxes(visible=False)
    thumbnail.layout.hovermode = 'closest'
    thumbnail.update_yaxes(scaleanchor = "x", scaleratio = 1)
    thumbnail.layout.showlegend = False
    thumbnail.layout.plot_bgcolor = 'white'
    thumbnail.layout.height=75
    thumbnail.layout.width=120
    
    with thumbnail.batch_update():
        thumbnail.add_trace(imagetodisplay)
        newcombo = widgets.VBox([localbutton,thumbnail])
        
        imagebank.children = imagebank.children + (newcombo,)
    
    
 
    
    
    
    
#    with lol.batch_update():
#        lol33.data = []
#        lol33.update_annotations(visible=False)
        
Uploader = FileUpload(multiple=False,description='Upload top or side view')
Uploader.observe(response, names='value')

In [ ]:
def changeimage(change):
    
    
    if change.owner.value == True:
        details1.value = 'Displaying ' +change.owner.description
    

In [ ]:
def printxy():
         details1.value = 'Touched'

In [ ]:
# create callback function for centers anchors
def update_point(trace, points, selector):
    
    global regions,scatter,  props
    #details1.value = "Clicked!"
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    n = len(c)
    # what out to the minus or plus 1 regarding position in list from 0 to n-1 
    # and the region numbering from 1 to n
    for i in points.point_inds:
        if c[i] == '#bae2be':
                
                c[i] = '#3BA7B3'
                s[i] = 10
                with lol.batch_update():
                    # i-1
                    #details0.value = "region <font size=5>"+str(i+1)+"</font> removed from the selection "
                    lol['data'][i]['line']['color']='#3BA7B3'
                    details1.value = "X-Z center position : <font size=5>"+str('{:.1f}'.format(regions[i].centroid[1]))+"</font>mm ,"+"<font size=5>"+str('{:.1f}'.format(-1*regions[i].centroid[0]))+"</font>mm    "
        else:
                if selectmultipleToggle.value == False:
                    foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
                    for j in range(len(foo_indexes)):
                        c[foo_indexes[j]] = '#3BA7B3'
                        s[foo_indexes[j]] = 10
                        with lol.batch_update():
                            scatter.marker.color = c
                            scatter.marker.size = s
                            lol['data'][foo_indexes[j]]['line']['color']='#3BA7B3'
                c[i] = '#bae2be'
                s[i] = 15
                with lol.batch_update():
                    #details0.value = "region <font size=5>"+str(i+1)+"</font> added to the selection "+"      Major lenght : <font size=5>"+ str('{:.0f}'.format(regions[i].major_axis_length))+"</font>mm"
                    lol['data'][i]['line']['color']='#bae2be'
                    details1.value = "X-Z center position : <font size=5>"+str('{:.1f}'.format(regions[i].centroid[1]))+"</font>mm ,"+"<font size=5>"+str('{:.1f}'.format(-1*regions[i].centroid[0]))+"</font>mm    "
        
    # find selected indices and get region number to display
    foo_indexes = [str(n+1) for n,x in enumerate(c) if x=='#bae2be']
    lst = ''
    for i in range(len(foo_indexes)):
            lst = lst + ' ' + foo_indexes[i] 
    # update graphical output   
    with lol.batch_update():
        scatter.marker.color = c
        scatter.marker.size = s
        details1.value = details1.value + "<p>Regions selected : <font size=5>" + lst +"</font>"

In [ ]:
# Callback fonction that save images to png and return completion into htlm text          
def response2(change):
    uploaded_filename = next(iter(Uploader2.value))
    content = Uploader2.value[uploaded_filename]['content']
    with open('myview2.png', 'wb') as f: f.write(content)
    text = Image.open('myview2.png')
        
    if Resize.value == True: # set the base width of the result
        basewidth = 300
        wpercent = (basewidth/float(text.size[0]))
        hsize = int((float(text.size[1])*float(wpercent)))
        # resize image
        text = text.resize((basewidth,hsize), Image.ANTIALIAS)
    else:
        pass
    
    if BlackandWhite.value == True: # set the base width of the result
        thresh = 254
        fn = lambda x : 255 if x > thresh else 0
        text = text.convert('L').point(fn, mode='1')
    else:
        pass

    
    pil_img = Image.fromarray(np.flipud(text)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
    with lol2.batch_update():
        lol2.data = []
        #details2.value = uploaded_filename+" has been uploaded."
        lol2.add_trace(go.Image(source = base64_string))
        details2.value = uploaded_filename+" has been uploaded"
        if Resize.value == True:
            details2.value = details2.value + ' and resized'
        details2.value = details2.value + '.'
#    with lol2.batch_update():
#        lol33.data = []
#        lol33.update_annotations(visible=False)
        
Uploader2 = FileUpload(multiple=False,description='Upload section')
Uploader2.observe(response2, names='value')

In [ ]:
#callback action to create region based on image
def segmenter(self):
    
    global regions,scatter, props
    
    text = io.imread('myview1.png')
    width = text.shape[:2][0]
    height = text.shape[:2][1]
    smooth = filters.gaussian(text, 1, preserve_range=True)
    #text_gray = rgb2gray(text) equivalent
    text_gray = np.dot(smooth[...,:3], [0.2989, 0.5870, 0.1140])
    thr = text_gray < 254
    anti = text_gray > 199
    mask = nd.morphology.binary_fill_holes(thr)
    mask = morphology.remove_small_objects(mask, 500)
    mask = morphology.remove_small_holes(mask, 500)


    lab, n = nd.label(anti*mask)
    # on fixe le nombre maximum de regions
    if n <= 3500:
        regions = regionprops(lab)
        props = regionprops_table(lab, properties=('centroid',
                                             'orientation',
                                             'major_axis_length',
                                             'minor_axis_length'))
        ###Contour detection using built in SCI-KIT library and trace 
        contours = find_contours(lab,1)
        # get the centroids coordinates from region properties
        centers = np.zeros((0, 2))
        for i in range(n):

            #also convert the tupples output into np.array
            a = np.asarray(regions[i].centroid).reshape(1,2)
            centers = np.append(centers,a,axis = 0)

        with lol.batch_update():

            details1.value = "Found <font size=5> "+str(len(centers))+"</font> regions et <font size=5> "+str(len(contours))+"</font> contours." 
            for index in range(1, lab.max()):

                    label = regions[index].label
                    contour = measure.find_contours(lab == label, 0.5)[0]
                    y, x = contour.T
                    hoverinfo = 'region '+str(label)
                    #for prop_name in properties:
                    #    hoverinfo += f'<b>{prop_name}: {getattr(props[index], prop_name):.2f}</b><br>'
                    lol.add_trace(go.Scatter(
                        x=x, y=-1*y+width, name=label,
                        mode='lines',line_color='#3BA7B3' , fill='toself',hovertemplate=hoverinfo, hoveron='points+fills'))

            colors = ['#3BA7B3'] *n
            sizes = [10] * n
            
            # add a trace containing the centroids + flip contours Y-axis to match the image Y-axis
            lol.add_trace(go.Scatter(x = centers[:, 1],
                                     y = -1*centers[:, 0]+width,
                                     mode = "markers",
                                     marker=dict(size=sizes,
                                                 color=colors)
                                    ))
            # scatter is the variable that contains the n regions centroid coordinates
            scatter = lol.data[n]
            scatter.marker.size = [10] * n
            scatter.marker.color = colors
            #details1.value = "COLOR!!!"
            scatter.on_click(update_point)

    else:
        with lol.batch_update():
            details1.value = "Error Found. <font size=5> "+str(n)+"</font> regions is too much. Please select another file." 

#segmentation button
Segment = widgets.Button(description = 'Segment')
Segment.on_click(segmenter)

In [ ]:
#callback action to create a skeleton of a region
def skeletonizer(self):
    global regions,scatter, props
    
    # find selected region
    c = list(scatter.marker.color)
    foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
        
    from skimage.morphology import skeletonize
    from skimage import data
    import matplotlib.pyplot as plt
    from skimage.util import invert

    image = regions[foo_indexes[0]].image
    
    # perform skeletonization
    skeleton = skeletonize(image)
    skeletoninvert = invert(skeleton)
    
    #full screen image
    pil_img = Image.fromarray(np.flipud(skeletoninvert*image)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
    
    imagetodisplay = go.Image(source = base64_string)
    #skeleton = skeletonize(base64_string)
    #skeletontodiplay = go.Image(source = skeleton)
    
    with lol2.batch_update():
        lol2.data = []
        #details1.value = uploaded_filename+" has been uploaded."
        lol2.add_trace(imagetodisplay)
        #lol.add_trace(skeletontodiplay)
        details1.value = "Region"+str(foo_indexes[0])+" has been skeletonized"
    
    
    
    
    #with lol.batch_update():
    #    lol.add_trace(go.Image(source = str(skeletoninvert*image*255)))
    
#segmentation button
Skeleton = widgets.Button(description = 'Skeleton')
Skeleton.on_click(skeletonizer)

In [ ]:
# Widgets

In [ ]:
# Html text widget for output

details1 = widgets.HTML("<font size=2>Nothing to analyse yet. Please feed informations.</font>")

In [ ]:
# Html text widget for output

details2 = widgets.HTML("<font size=2>Nothing to analyse yet. Please feed informations.</font>")

In [ ]:
# Resize image  if True
Resize = widgets.ToggleButton(value = False,description = 'Resize')

In [ ]:
# Select Multiple Regions
selectmultipleToggle = widgets.ToggleButton(
                            value=False,
                            description='SelectMultiple',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Toggle between single and multiple selection',
                            icon='plus' # (FontAwesome names without the `fa-` prefix)
                        )


In [ ]:
# black and white if True
BlackandWhite = widgets.ToggleButton(value = False,description = 'B&W')

In [ ]:
# Image bank display

layoutfull = Layout(overflow_x='hidden',
                    overflow_y='hidden',
                    border='3px solid #F3EFE0',
                    #width='480px',
                    height='125px',
                    flex_flow='row',
                    #display='flex'
                   )

imagebank = widgets.VBox([],layout = layoutfull)
#imagebank

In [ ]:
# Figure with layout for display

layout = go.Layout(title='',
                    #width=480,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=10,
                                pad=4
                                ),
                   )



lol = go.FigureWidget(layout=layout)

# define root figure layout
lol.update_xaxes(autorange=True)
lol.update_yaxes(autorange=True)
lol.update_xaxes(visible=False)  
lol.update_yaxes(visible=False)
lol.layout.hovermode = 'closest'
lol.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol.layout.showlegend = False
lol.layout.plot_bgcolor = 'white'
lol.layout.width=400



In [ ]:
# Figure for SECTION display

lol2 = go.FigureWidget(layout=layout)

# define root figure layout
lol2.update_xaxes(autorange=True)
lol2.update_yaxes(autorange=True)
lol2.update_xaxes(visible=False)  
lol2.update_yaxes(visible=False)
lol2.layout.hovermode = 'closest'
lol2.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol2.layout.showlegend = False
lol2.layout.plot_bgcolor = 'white'
lol2.layout.width=400


boxlayout = Layout(width='1080',
                   align_items='stretch',
                   #justify_items = 'flex-start',
                   justify_items = 'center',
                   #display='flex',
                   flex_flow = 'column',
                   #align_content='stretch'
                  )

flouplayout = Layout(width='1080',
                   align_items='stretch',
                   justify_items = 'flex-start',
                   #display='flex',
                   flex_flow = 'line',
                   #align_content='stretch'
                  )

In [ ]:
# Initiate variables
listofimage = []




In [ ]:
display = widgets.HBox([lol,lol2],layout=flouplayout )
planview = widgets.VBox([details1,display])
#sectionview = widgets.VBox([Uploader2,details2,lol2])
Bandeau = widgets.HBox([Resize, BlackandWhite,Uploader,selectmultipleToggle,Segment,Skeleton])
Main = widgets.VBox([planview,imagebank],layout=boxlayout)
app = widgets.VBox([Bandeau,Main],layout=boxlayout)

# App

In [ ]:
app